In [97]:
import pandas as pd
import numpy as np

In [98]:
spacep = pd.read_excel('../file/空间类型表.xlsx')

In [99]:
spacep

,Unnamed: 0,房间组合,房间编号,sound Insulation（1较好 0适中 -1不隔声）,Facility (1Yes 0No),Spatial properties（1Open 2Close),Flexible(F) or Fix(X),Refurniture (1Yes 0No),Accommodate events,容纳人数,开放时长
0,normal space,C1,C1-1,0,0,2.0,F,1,交流/会议/自习,8,07:00-22:00
1,NaN,NaN,C1-2,0,0,2.0,F,1,交流/会议/自习,8,07:00-22:00
2,NaN,NaN,C1-3,0,1| Projector,2.0,F,1,交流/会议/自习,14,07:00-22:00
3,NaN,C2,C2-1,0,1| Projector,2.0,F,1,交流/会议/自习,8,07:00-22:00
4,NaN,NaN,C2-2,0,0,2.0,F,1,交流/会议/自习,8,07:00-22:00
...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,fix seat,S1',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00
90,NaN,NaN,S2',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00
91,NaN,NaN,S1'',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00
92,NaN,NaN,S2'',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00


In [100]:
# spacep.rename({'sound Insulation（1较好 0适中 -1不隔声）':'','':'sp'})
spacep.columns=['0','房间组合','房间编号','soundi','facility','sproperty','flex','refur','events','num','time']

In [101]:
spacep

,0,房间组合,房间编号,soundi,facility,sproperty,flex,refur,events,num,time
0,normal space,C1,C1-1,0,0,2.0,F,1,交流/会议/自习,8,07:00-22:00
1,NaN,NaN,C1-2,0,0,2.0,F,1,交流/会议/自习,8,07:00-22:00
2,NaN,NaN,C1-3,0,1| Projector,2.0,F,1,交流/会议/自习,14,07:00-22:00
3,NaN,C2,C2-1,0,1| Projector,2.0,F,1,交流/会议/自习,8,07:00-22:00
4,NaN,NaN,C2-2,0,0,2.0,F,1,交流/会议/自习,8,07:00-22:00
...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,fix seat,S1',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00
90,NaN,NaN,S2',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00
91,NaN,NaN,S1'',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00
92,NaN,NaN,S2'',1,1| Computer,NaN,X,1,固定工位,20,07:00-22:00


### 处理facility

In [102]:
# 数据映射表
facility_mapping = {
    '0':0,
    '1| Projector':1,
    '1| Board':2,
    '1| Computer':3,
    '1| Shelves':4,
    '1| 3D printer':5,
    '1| Eye tracker':6,
    '1| EEG':7,
    '1| Audio-visual equipment':8
}
# 根据映射修改列
spacep['facility'] = spacep['facility'].apply(lambda x: facility_mapping.get(x, 0)) #若存在key则映射，不能则0

# '1|add'.split('|')  # ['1', 'add']
# '0'.split('|') ##0

### 处理flex

In [103]:
flexibility_mapping = {
    'F':1,
    'X':0
} # F表示灵活，X表示固定
spacep['flex'] = spacep['flex'].apply(lambda x: flexibility_mapping.get(x))

### 处理soundi

In [104]:
# 映射
soundi_mapping ={
    -1:0,
     0:1,
     1:2,
} #不隔声0，一般1，隔声2
spacep['soundi'] = spacep['soundi'].apply(lambda x: soundi_mapping.get(x))

### 处理events

In [105]:
events = set() #用于存储不重复存储事件
for row in spacep['events']:
    events.update(row.split('/')) 

# 判断事件是否存在并为该条数据赋值
for event in events:
    spacep[event] = spacep['events'].apply(lambda x: 1 if event in x else 0)

### 处理spatial property

In [106]:
spacep['sproperty']

0     2.0
1     2.0
2     2.0
3     2.0
4     2.0
     ... 
89    NaN
90    NaN
91    NaN
92    NaN
93    NaN
Name: sproperty, Length: 94, dtype: float64

In [107]:
# 映射
sproperty_mapping ={
     2.:0,
     1.:1,
} #1open,0close,nan无要求
spacep['sproperty'] = spacep['sproperty'].apply(lambda x: sproperty_mapping.get(x))

In [108]:
spacep.sproperty[2],spacep.sproperty[60],spacep.sproperty[90]

(0.0, 1.0, nan)

In [109]:
spacep

,0,房间组合,房间编号,soundi,facility,sproperty,flex,refur,events,num,...,上课,实验,会议,报告,固定工位,评图,办公,交流,模型,咖啡
0,normal space,C1,C1-1,1,0,0.0,1,1,交流/会议/自习,8,...,0,0,1,0,0,0,0,1,0,0
1,NaN,NaN,C1-2,1,0,0.0,1,1,交流/会议/自习,8,...,0,0,1,0,0,0,0,1,0,0
2,NaN,NaN,C1-3,1,1,0.0,1,1,交流/会议/自习,14,...,0,0,1,0,0,0,0,1,0,0
3,NaN,C2,C2-1,1,1,0.0,1,1,交流/会议/自习,8,...,0,0,1,0,0,0,0,1,0,0
4,NaN,NaN,C2-2,1,0,0.0,1,1,交流/会议/自习,8,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,fix seat,S1',2,3,NaN,0,1,固定工位,20,...,0,0,0,0,1,0,0,0,0,0
90,NaN,NaN,S2',2,3,NaN,0,1,固定工位,20,...,0,0,0,0,1,0,0,0,0,0
91,NaN,NaN,S1'',2,3,NaN,0,1,固定工位,20,...,0,0,0,0,1,0,0,0,0,0
92,NaN,NaN,S2'',2,3,NaN,0,1,固定工位,20,...,0,0,0,0,1,0,0,0,0,0


In [110]:
spacep.describe()

,soundi,facility,sproperty,flex,refur,num,阅览,自习,展示,上课,实验,会议,报告,固定工位,评图,办公,交流,模型,咖啡
count,94.000000,94.000000,74.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.00000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.00000,94.000000,94.000000,94.000000
mean,1.361702,1.319149,0.202703,0.585106,0.680851,20.851064,0.010638,0.787234,0.06383,0.297872,0.042553,0.670213,0.010638,0.042553,0.414894,0.06383,0.489362,0.042553,0.031915
std,0.745612,1.844566,0.404757,0.495346,0.468646,15.150824,0.103142,0.411458,0.24576,0.459775,0.202930,0.472657,0.103142,0.202930,0.495346,0.24576,0.502567,0.202930,0.176716
min,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,1.000000,14.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,0.000000,1.000000,1.000000,30.000000,0.000000,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.00000,1.000000,0.000000,0.000000
max,2.000000,8.000000,1.000000,1.000000,1.000000,70.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [111]:
spacep.to_excel('./spacep.xlsx')